# **DATA PREPARATION**

## **IMPORT LIBRARY**

In [2]:
import pandas as pd

## **DATASET**

In [3]:
file = '/content/supermarket_sales new.csv'

df = pd.read_csv(file)
df

,Gender,Invoice ID,Branch,City,Customer type,Product line,Unit price,Quantity,Tax 5%
0,Female,750-67-8428,A,Yangon,Member,Health and beauty,74.69,7,261.4150
1,Female,226-31-3081,C,Naypyitaw,Normal,Electronic accessories,15.28,5,3.8200
2,Female,355-53-5943,A,Yangon,Member,Electronic accessories,68.84,6,20.6520
3,Female,315-22-5665,C,Naypyitaw,Normal,Home and lifestyle,73.56,10,36.7800
4,Female,665-32-9167,A,Yangon,Member,Health and beauty,36.26,2,3.6260
...,...,...,...,...,...,...,...,...,...
995,Male,745-74-0715,A,Yangon,Normal,Electronic accessories,58.03,2,5.8030
996,Male,690-01-6631,B,Mandalay,Normal,Fashion accessories,17.49,10,8.7450
997,Male,233-67-5758,C,Naypyitaw,Normal,Health and beauty,40.35,1,2.0175
998,Male,727-02-1313,A,Yangon,Member,Food and beverages,31.84,1,1.5920


## **Data Cleaning**

### **1. Tipe Data**
Tipe data pada setiap variabel atau kolom sudah sesuai dengan fungsinya.

In [11]:
df.dtypes

,0
Gender,object
Invoice ID,object
Branch,object
City,object
Customer type,object
Product line,object
Unit price,float64
Quantity,int64
Tax 5%,float64


In [10]:
df['Invoice ID'] = df['Invoice ID'].astype(str)

### **2. Incosistent Values**

Tahap ini melakukan penanganan pada data yang tidak sesuai format. Dari dataset marker sales data ini sudah konsisten secara format.

### **3. Missing Values**

Berdasarkan hasil pengecekan menggunakan fungsi kode dibawah, seluruh kolom dalam dataset memiliki tingkat kelengkapan 100% (0.00 Missing Values). Oleh karena itu, tidak diperlukan tindakan penghapusan baris maupun imputasi

In [29]:
pd.DataFrame(df.isna().sum() / len(df) * 100, columns=['Null Ratio in %'])

,Null Ratio in %
Gender,0.00
Invoice ID,0.00
Branch,0.00
City,0.00
Customer type,0.00
Product line,0.00
Unit price,0.00
Quantity,0.00
Tax 5%,0.00


### **4. Duplicated values**

Hasil pengecekan menunjukkan 0 (Nol) Duplicate Values. Artinya, tidak ditemukan baris data yang identik secara keseluruhan maupun duplikasi. Setiap baris dalam dataset ini mewakili satu transaksi yang unik dan berbeda

In [30]:
df[df.duplicated()]

,Gender,Invoice ID,Branch,City,Customer type,Product line,Unit price,Quantity,Tax 5%


### **5. Outliers**

Berdasarkan deteksi outlier, kolom Unit price dan Quantity tidak memiliki outlier (0%). Namun, pada kolom Tax 5% ditemukan outlier sebesar 10.9%. Sesuai aturan penanganan, karena persentase > 10% dan mempertimbangkan konteks bahwa outlier tersebut merupakan anomali input (percampuran data), maka dilakukan Imputasi dengan metode Capping. Hal ini dilakukan agar data ekstrem tidak merusak distribusi statistik namun tetap mempertahankan jumlah sampel data

In [31]:
columns_to_impute = ["Tax 5%"]

for col in columns_to_impute:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df.loc[:, col] = df[col].clip(lower=lower_bound, upper=upper_bound)



**Hasil:**

In [34]:
results = []
col = "Tax 5%"

Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
percent_outliers = (len(outliers) / len(df)) * 100
results.append({
    'Kolom': col,
    'Jumlah Outliers': len(outliers),
    'Persentase Outliers (%)': round(percent_outliers, 2)
})
results_df = pd.DataFrame(results)
results_df.set_index('Kolom', inplace=True)
display(results_df)

,Jumlah Outliers,Persentase Outliers (%)
Kolom,,
Tax 5%,0,0.00


## **Construct Data**

* Total Before Tax: Dasar nilai transaksi sebelum penambahan pajak. Menambahkan kolom ini untuk mengetahui Nilai Transaksi Murni.

* Total Sales: Dibuat untuk mencatat total pembayaran akhir dari pelanggan, dan menunjukkan pendapatan kotor yang sebenarnya.

In [55]:
df['Total Before Tax'] = df['Unit price'] * df['Quantity']

df['Total Sales'] = df['Total Before Tax'] + df['Tax 5%']


display(df[['Total Before Tax', 'Total Sales']].head())


,Total Before Tax,Total Sales
0,522.83,581.56
1,76.40,80.22
2,413.04,433.69
3,735.60,772.38
4,72.52,76.15


In [56]:
df

,Gender,Invoice ID,Branch,City,Customer type,Product line,Unit price,Quantity,Tax 5%,Total Before Tax,Total Sales
0,Female,750-67-8428,A,Yangon,Member,Health and beauty,74.69,7,58.73,522.83,581.56
1,Female,226-31-3081,C,Naypyitaw,Normal,Electronic accessories,15.28,5,3.82,76.40,80.22
2,Female,355-53-5943,A,Yangon,Member,Electronic accessories,68.84,6,20.65,413.04,433.69
3,Female,315-22-5665,C,Naypyitaw,Normal,Home and lifestyle,73.56,10,36.78,735.60,772.38
4,Female,665-32-9167,A,Yangon,Member,Health and beauty,36.26,2,3.63,72.52,76.15
...,...,...,...,...,...,...,...,...,...,...,...
995,Male,745-74-0715,A,Yangon,Normal,Electronic accessories,58.03,2,5.80,116.06,121.86
996,Male,690-01-6631,B,Mandalay,Normal,Fashion accessories,17.49,10,8.74,174.90,183.64
997,Male,233-67-5758,C,Naypyitaw,Normal,Health and beauty,40.35,1,2.02,40.35,42.37
998,Male,727-02-1313,A,Yangon,Member,Food and beverages,31.84,1,1.59,31.84,33.43


## **Data Reduction**

Pada tahap ini, mengurangi fitur kolom yang tidak berguna atau tidak memberikan informasi penting. Tujuannya agar dataset menjadi lebih ringan, rapi, dan bisa fokus pada data yang benar benar perlu di analisis.


* **Invoice ID:** hanyalah kode unik (seperti nomor struk) yang berbeda untuk setiap orang. Kode ini tidak bisa dihitung (tidak bisa dijumlahkan atau dirata-rata) dan tidak memiliki pola.

In [57]:
df = df.drop('Invoice ID', axis=1)

In [58]:
df

,Gender,Branch,City,Customer type,Product line,Unit price,Quantity,Tax 5%,Total Before Tax,Total Sales
0,Female,A,Yangon,Member,Health and beauty,74.69,7,58.73,522.83,581.56
1,Female,C,Naypyitaw,Normal,Electronic accessories,15.28,5,3.82,76.40,80.22
2,Female,A,Yangon,Member,Electronic accessories,68.84,6,20.65,413.04,433.69
3,Female,C,Naypyitaw,Normal,Home and lifestyle,73.56,10,36.78,735.60,772.38
4,Female,A,Yangon,Member,Health and beauty,36.26,2,3.63,72.52,76.15
...,...,...,...,...,...,...,...,...,...,...
995,Male,A,Yangon,Normal,Electronic accessories,58.03,2,5.80,116.06,121.86
996,Male,B,Mandalay,Normal,Fashion accessories,17.49,10,8.74,174.90,183.64
997,Male,C,Naypyitaw,Normal,Health and beauty,40.35,1,2.02,40.35,42.37
998,Male,A,Yangon,Member,Food and beverages,31.84,1,1.59,31.84,33.43
